# アソシエーション分析の例
- 全体の流れ
  - 少し偏りのあるデータセットを作成。
  - orange3-associate パッケージによりアソシエーション分析。
- 参考
  - [Python でアソシエーション分析 - Orange3-Associate](https://fits.hatenablog.com/entry/2018/01/09/211900)

## 環境構築

In [1]:
!pip install orange3
!pip install orange3-associate

## データセット作成

In [2]:
import random

def make_dataset(num_sample=100, num_item=15):
  """データセット構築。
  一度に購入した商品軍をリストとして作成。
  偏り（傾向）を持たせるため、特定条件に合致する場合は別商品も購入するように設定。

  num_sample (int): サンプル数。
  num_item (int)  : 商品種類数。
  """
  dataset = []
  for i in range(num_sample):
    temp = []
    size = random.randint(1, 3)
    for j in range(size):
      item = random.randint(1, num_item)
      temp.append(item)
      if item == 1:      # 特別条件1
        temp.append(10)  # 　商品1を購入した場合は、商品10も購入する。
      elif item == 2:    # 特別条件2(商品2を購入した場合は、70%の確率で商品3,4も購入する。)
        if random.random() <= 0.7:
          temp.append(3)
          temp.append(4)
    dataset.append(temp[:])
  return dataset

dataset = make_dataset(num_sample=100)
print(dataset[0:5])

[[5, 9, 2, 3, 4], [8], [8, 4], [3], [9, 14]]


## データセットをCSVファイルとして保存

In [3]:
def save_dataset(dataset, filename="data.basket"):
  with open(filename, "w") as f:
    for items in dataset:
      line = ""
      for index, value in enumerate(items):
        if index == 0:
          line += str(value)
        else:
          line += "," + str(value)
      line += "\n"
      f.write(line)

filename = "data.basket"
save_dataset(dataset, filename)
!head data.basket

5,9,2,3,4
8
8,4
3
9,14
15
15,6
14
11
11,2,3,4


## アソシエーション分析
- モジュール読み込み後、Orange.data.Table()によりCSVファイルを読み込む。各商品を OneHoe.encode によりone-hotエンコードする。
- frequent_itemsets() によりカウント。第2引数 min_support は、抽出件数最小値を設定。（比率でも設定できる模様）
- association_rules() によりアソシエーションルールの抽出。第2引数 min_conficende は、抽出する confidence の最小値を指定。

In [4]:
import Orange
from orangecontrib.associate.fpgrowth import *

In [5]:
# データファイル読み込み
tbl = Orange.data.Table(filename)

X, mapping = OneHot.encode(tbl)

itemsets = dict(frequent_itemsets(X, 5))
print(X[:5])
print(itemsets)

[[0, 1, 2, 3, 4], [5], [4, 5], [3], [1, 6]]
{frozenset({0}): 13, frozenset({1}): 14, frozenset({2}): 11, frozenset({3}): 21, frozenset({2, 3}): 8, frozenset({4}): 22, frozenset({1, 4}): 5, frozenset({2, 4}): 9, frozenset({3, 4}): 8, frozenset({2, 3, 4}): 8, frozenset({5}): 10, frozenset({6}): 19, frozenset({7}): 16, frozenset({8}): 16, frozenset({9}): 13, frozenset({9, 6}): 5, frozenset({10}): 10, frozenset({11}): 14, frozenset({12}): 11, frozenset({13}): 12, frozenset({14}): 24, frozenset({3, 14}): 5, frozenset({4, 14}): 6, frozenset({13, 14}): 12}


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
# アソシエーションルールの抽出
# 第2引数: min_confidence は確信度の最小値
rules = association_rules(itemsets, 0.2)

In [7]:
def decode_onehot(d):
    items = OneHot.decode(d, tbl, mapping)
    # ContinuousVariable の name 値を取得
    return list(map(lambda v: v[1].name, items))

for P, Q, support, confidence in rules:
  lhs = decode_onehot(P)
  rhs = decode_onehot(Q)
  print(f"lhs = {lhs}, rhs = {rhs}, support = {support}, confidence = {confidence}")


lhs = ['3', '4'], rhs = ['2'], support = 8, confidence = 1.0
lhs = ['2', '4'], rhs = ['3'], support = 8, confidence = 0.8888888888888888
lhs = ['4'], rhs = ['2', '3'], support = 8, confidence = 0.36363636363636365
lhs = ['2', '3'], rhs = ['4'], support = 8, confidence = 1.0
lhs = ['3'], rhs = ['2', '4'], support = 8, confidence = 0.38095238095238093
lhs = ['2'], rhs = ['3', '4'], support = 8, confidence = 0.7272727272727273
lhs = ['3'], rhs = ['2'], support = 8, confidence = 0.38095238095238093
lhs = ['2'], rhs = ['3'], support = 8, confidence = 0.7272727272727273
lhs = ['4'], rhs = ['9'], support = 5, confidence = 0.22727272727272727
lhs = ['9'], rhs = ['4'], support = 5, confidence = 0.35714285714285715
lhs = ['4'], rhs = ['2'], support = 9, confidence = 0.4090909090909091
lhs = ['2'], rhs = ['4'], support = 9, confidence = 0.8181818181818182
lhs = ['4'], rhs = ['3'], support = 8, confidence = 0.36363636363636365
lhs = ['3'], rhs = ['4'], support = 8, confidence = 0.38095238095238093